---
# Applied Social Network Analysis
We have performed three tasks under this project:
* Analyse the algorithm by which a set of randomly generated graphs are constructed i.e, Preferential Attachment/Small World
* Predict whether or an employee in a company will receive a management position salary.
* Predict the future possible connections between nodes in a network.

In [14]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---
## Random Graph Identification

For this part we analyzed randomly generated graphs and determine which algorithm created them
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

In [15]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

In [16]:
def degree_distribution(G):
    l=G.degree()
    degree = sorted(set(l.values()))
    histogram = [list(l.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree]
    return histogram
for G in P1_Graphs:
    print(degree_distribution(G))

[0.465, 0.237, 0.1, 0.056, 0.032, 0.026, 0.016, 0.014, 0.009, 0.007, 0.005, 0.005, 0.002, 0.001, 0.001, 0.002, 0.002, 0.003, 0.002, 0.001, 0.002, 0.002, 0.002, 0.001, 0.002, 0.001, 0.002, 0.001, 0.001]
[0.002, 0.023, 0.166, 0.615, 0.171, 0.022, 0.001]
[0.0026666666666666666, 0.124, 0.756, 0.10533333333333333, 0.012]
[0.3453333333333333, 0.17733333333333334, 0.11866666666666667, 0.07733333333333334, 0.06266666666666666, 0.03333333333333333, 0.025333333333333333, 0.017333333333333333, 0.02, 0.018666666666666668, 0.010666666666666666, 0.009333333333333334, 0.006666666666666667, 0.012, 0.005333333333333333, 0.0013333333333333333, 0.0013333333333333333, 0.005333333333333333, 0.008, 0.0013333333333333333, 0.0026666666666666666, 0.0013333333333333333, 0.008, 0.004, 0.0013333333333333333, 0.0026666666666666666, 0.0026666666666666666, 0.0013333333333333333, 0.0026666666666666666, 0.0026666666666666666, 0.004, 0.0026666666666666666, 0.0013333333333333333, 0.0013333333333333333, 0.001333333333333

In [17]:
def arry(graphs):
    arr=[]
    for G in graphs:
        arr.append([nx.average_clustering(G),nx.average_shortest_path_length(G),len(degree_distribution(G))])
    return arr
arry(P1_Graphs)

[[0.03167539146454044, 4.099161161161161, 29],
 [0.5642419635919628, 5.089871871871872, 7],
 [0.4018222222222227, 9.378702269692925, 5],
 [0.03780379975223251, 3.1048046283934134, 37],
 [0.0033037037037037037, 5.0785509568313305, 9]]

In [18]:
def graph_identification():
    a=[]
    arr=arry(P1_Graphs)
    for i in range(5):
        if arr[i][2]>10:
            a.append('PA')
        elif arr[i][0]<0.1:
            a.append('SW_H')
        else:
            a.append('SW_L')
    return a
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Salary Prediction

For this part we worked with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`. 

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [19]:
G = nx.read_gpickle('email_prediction.txt')
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (2, {'Department': 21, 'ManagementSalary': nan}),
 (3, {'Department': 21, 'ManagementSalary': 1.0}),
 (4, {'Department': 21, 'ManagementSalary': 1.0}),
 (5, {'Department': 25, 'ManagementSalary': nan}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (7, {'Department': 14, 'ManagementSalary': 0.0}),
 (8, {'Department': 14, 'ManagementSalary': nan}),
 (9, {'Department': 14, 'ManagementSalary': 0.0}),
 (10, {'Department': 9, 'ManagementSalary': 0.0}),
 (11, {'Department': 14, 'ManagementSalary': 0.0}),
 (12, {'Department': 14, 'ManagementSalary': 1.0}),
 (13, {'Department': 26, 'ManagementSalary': 1.0}),
 (14, {'Department': 4, 'ManagementSalary': nan}),
 (15, {'Department': 17, 'ManagementSalary': 0.0}),
 (16, {'Department': 34, 'ManagementSalary': 0.0}),
 (17, {'Department': 1, 'ManagementSalary': 0.0}),
 (18, {'Department': 1, 'ManagementSalary': nan}),
 (19, {'Department': 14, 'Ma

---

Using network `G`, we identified the people in the network with missing values for the node attribute `ManagementSalary` and predicted whether or not these individuals are receiving a management position salary.

To accomplish this we trained a sklearn classifier on nodes that have `ManagementSalary` data, and predicted a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.


In [20]:
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

def salary_predictions():
    def ismanagement(node):
        return node[1]['ManagementSalary']
    df = pd.DataFrame(index= G.nodes())
    df['Clustering'] = pd.Series(nx.clustering(G))
    df['Degree'] = pd.Series(nx.degree(G))
    df['Degree Centrality'] = pd.Series(nx.degree_centrality(G))
    df['Closeness'] = pd.Series(nx.closeness_centrality(G))
    df['Betweenness'] = pd.Series(nx.betweenness_centrality(G))
    df['PageRank'] = pd.Series(nx.pagerank(G))
    df['Management'] = pd.Series([ismanagement(node) for node in G.nodes(data=True)])
    df_train = df[~pd.isnull(df['Management'])]
    df_test = df[pd.isnull(df['Management'])]
    features = ['Clustering', 'Degree', 'Degree Centrality', 'Closeness', 'Betweenness', 'PageRank']
    X_train = df_train[features]
    Y_train = df_train['Management']
    X_test = df_test[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5,
                       random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scaled, Y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:,1]
    return pd.Series(test_proba, X_test.index)
salary_predictions()

1       0.128439
2       0.586261
5       0.979859
8       0.133702
14      0.304625
18      0.208766
27      0.269586
30      0.336144
31      0.169191
34      0.143513
37      0.096221
40      0.195036
45      0.162267
54      0.249954
55      0.252905
60      0.176908
62      0.992842
65      0.551284
77      0.078478
79      0.132108
97      0.099812
101     0.057517
103     0.214342
108     0.122930
113     0.454489
122     0.044350
141     0.396296
142     0.863703
144     0.072049
145     0.419966
          ...   
913     0.045734
914     0.078520
915     0.002087
918     0.144304
923     0.035175
926     0.090079
931     0.055969
934     0.013933
939     0.011147
944     0.002353
945     0.039608
947     0.162715
950     0.098173
951     0.038313
953     0.027345
959     0.002922
962     0.002440
963     0.139886
968     0.110279
969     0.104672
974     0.095061
984     0.012668
987     0.110375
989     0.113364
991     0.116944
992     0.003956
994     0.003077
996     0.0024

---
### New Connections Prediction

In this part, we predicted the future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. 
The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.


To accomplish this we trained a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.


In [43]:
future_connections = pd.read_csv('Future_Connections.csv',index_col=0, converters={0: eval})
future_connections.head(10)

def new_connections_predictions():
    for node in G.nodes():
            G.node[node]['community'] = G.node[node]['Department']
            
    preferential_attachment = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(x[0], x[1]) for x in preferential_attachment])
    df['preferential_attachment'] = [x[2] for x in preferential_attachment]
    
    cns = list(nx.cn_soundarajan_hopcroft(G))
    df['cns'] = [x[2] for x in cns]
    
    df['resource_allocation_index'] = [x[2] for x in list(nx.resource_allocation_index(G))]
    
    df['jaccard_coefficient'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    
    df = future_connections.join(df,how='outer')
    
    df_train = df[~pd.isnull(df['Future Connection'])]
    df_test = df[pd.isnull(df['Future Connection'])]
    
    features = ['cns', 'preferential_attachment', 'resource_allocation_index', 'jaccard_coefficient']
    
    X_train = df_train[features]
    Y_train = df_train['Future Connection']
    X_test = df_test[features]
    
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    clf = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5,random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scaled, Y_train)
    
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    predictions = pd.Series(test_proba,X_test.index)
    
    null_values = future_connections[pd.isnull(df['Future Connection'])]
    null_values['prob'] = [predictions[x] for x in target.index]
    return null_values['prob']

new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.030798
(542, 751)    0.012661
(20, 426)     0.590189
(50, 989)     0.012774
(942, 986)    0.012896
(324, 857)    0.012758
(13, 710)     0.157992
(19, 271)     0.107565
(319, 878)    0.012813
(659, 707)    0.012673
(49, 843)     0.012848
(208, 893)    0.012700
(377, 469)    0.008606
(405, 999)    0.021212
(129, 740)    0.018759
(292, 618)    0.023529
(239, 689)    0.012821
(359, 373)    0.010155
(53, 523)     0.076263
(276, 984)    0.012843
(202, 997)    0.012866
(604, 619)    0.047400
(270, 911)    0.012819
(261, 481)    0.063398
(200, 450)    0.997548
(213, 634)    0.012626
(644, 735)    0.041949
(346, 553)    0.012347
(521, 738)    0.011817
(422, 953)    0.019543
                ...   
(672, 848)    0.012819
(28, 127)     0.977766
(202, 661)    0.012300
(54, 195)     0.997570
(295, 864)    0.012741
(814, 936)    0.012643
(839, 874)    0.012896
(139, 843)    0.012750
(461, 544)    0.011453
(68, 487)     0.011390
(622, 932)    0.012713
(504, 936)    0.017593
(479, 528) 